# Restricted Boltzmann Machines on MNIST

In [2]:
# importing relevant libraries
import torch
import numpy as np

In [35]:
class RBM:
    
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible   # number of visible units in RBM
        self.num_hidden = num_hidden    # number of hidden units in RBM
        
        np_rand = np.random.RandomState(1234)
        
        # create a random spread of weights based on a normal distribution
        self.weights = np.asarray(np_rand.uniform(
                                    low=-0.1 * np.sqrt(6. / (num_visible + num_hidden)),
                                    high=0.1 * np.sqrt(6. / (num_visible + num_hidden)),
                                    size=(num_visible, num_hidden)))
        
        # adding bias along axis 0, i.e adding a row of 0's before all the weights
        # this is the bias for hidden neurons
        self.weights = np.insert(self.weights, 0, 0, axis=0)
        
        # adding bias along axis 1, i.e adding a new column before 1st column in the weights
        # this is the bias for the visible neurons
        self.weights = np.insert(self.weights, 0, 0, axis=1)
        
    def train(self, data, max_epochs = 1000, alpha=)

In [33]:
r = RBM(784, 25)
wts = r.weights
wts

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.31323648e-03,  2.10318861e-03, -1.07257086e-03, ...,
        -7.31358875e-03, -2.25936150e-03,  7.46035948e-03],
       [ 2.60732119e-03, -1.77057197e-03,  4.97305756e-03, ...,
         1.05811690e-03, -2.93377275e-03,  5.11004209e-05],
       ...,
       [ 5.85584252e-03,  5.13815751e-03,  7.29254497e-03, ...,
        -1.71656446e-03, -7.87590787e-03, -6.66450268e-03],
       [-2.95153574e-03, -7.66219311e-03,  6.40225600e-03, ...,
        -2.23265559e-03,  5.25319795e-03,  3.69438135e-03],
       [-2.70736160e-03,  3.64618761e-03, -7.58282809e-03, ...,
         5.35289213e-04, -6.54133918e-03, -5.58677335e-03]])